<a href="https://colab.research.google.com/github/Chandan-devraj/Capstone-Project/blob/main/Indian_faces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests opencv-python-headless mediapipe pillow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.1/36.1 MB 46.4 MB/s eta 0:00:00


In [ ]:
import os
import requests
import cv2
import numpy as np
from PIL import Image
from io import BytesIO
import mediapipe as mp


In [ ]:
# Pixabay API Key
PIXABAY_API_KEY = "27542215-1bcc2dba5b7791b910e679866"

# Mediapipe Setup
mp_face_detection = mp.solutions.face_detection
mp_face_mesh = mp.solutions.face_mesh

# Directory to store valid images
os.makedirs("3", exist_ok=True)


In [ ]:
def fetch_images(query, num_images=50):
    """Fetch images from Pixabay API."""
    url = f"https://pixabay.com/api/?key={PIXABAY_API_KEY}&q={query}&image_type=photo&per_page={num_images}"
    response = requests.get(url).json()
    return response.get('hits', [])


In [ ]:
def is_valid_face(image_url):
    """Check if the image has a single, straight-looking face with both eyes, nose, and mouth visible."""
    try:
        # Download the image
        response = requests.get(image_url)
        image = Image.open(BytesIO(response.content)).convert("RGB")
        image_np = np.array(image)  # Convert to NumPy array

        # Initialize Mediapipe face detection
        with mp_face_detection.FaceDetection(min_detection_confidence=0.5) as face_detection:
            results = face_detection.process(cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR))

            # Check if exactly one face is detected
            if not results.detections or len(results.detections) != 1:
                return None  # Invalid image

        # Initialize Mediapipe face mesh for landmarks
        with mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, refine_landmarks=True) as face_mesh:
            results = face_mesh.process(cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR))

            # Check if landmarks are detected
            if not results.multi_face_landmarks:
                return None  # Invalid image

            # Extract landmarks
            landmarks = results.multi_face_landmarks[0]
            landmark_points = landmarks.landmark

            # Ensure all landmarks are detected (we expect 468 landmarks)
            if len(landmark_points) < 468:
                return None  # Not all landmarks detected

            # Check visibility of key facial features: both eyes, nose, and mouth
            left_eye = landmark_points[33]  # Left eye index
            right_eye = landmark_points[263]  # Right eye index
            nose_tip = landmark_points[1]  # Nose tip index
            mouth_left = landmark_points[13]  # Left mouth corner
            mouth_right = landmark_points[14]  # Right mouth corner

            # Check that both eyes are visible
            if left_eye.visibility < 0.5 or right_eye.visibility < 0.5:
                return None  # At least one eye is not visible

            # Check that the nose is visible
            if nose_tip.visibility < 0.5:
                return None  # Nose is not visible

            # Check that the mouth is visible
            if mouth_left.visibility < 0.5 or mouth_right.visibility < 0.5:
                return None  # Mouth is not visible

            # Ensure eyes are horizontally aligned (for a straight face)
            if abs(left_eye.y - right_eye.y) > 0.05:  # Tolerance for alignment
                return None  # Face not straight-looking

            # If all checks pass, return the valid image
            return image

    except Exception as e:
        print(f"Error processing image: {e}")
    return None


In [ ]:
os.makedirs("7", exist_ok=True)

In [ ]:
def process_images(query, num_images=50):
    """Fetch and process images."""
    images = fetch_images(query, num_images)
    valid_count = 0

    for i, img_data in enumerate(images):
        img_url = img_data.get('webformatURL')
        if img_url:
            valid_image = is_valid_face(img_url)
            if valid_image:
                valid_count += 1
                valid_image.save(f"7/face_{valid_count}.jpg")
                print(f"Saved: face_{valid_count}.jpg")

    print(f"Total valid images saved: {valid_count}")

# Run the process
process_images("Portrait , straight lokking single Indian face with eyes nose nd mouth clearly visible", num_images=50)


Saved: face_1.jpg
Saved: face_2.jpg
Saved: face_3.jpg
Saved: face_4.jpg
Saved: face_5.jpg
Saved: face_6.jpg
Saved: face_7.jpg
Saved: face_8.jpg
Saved: face_9.jpg
Saved: face_10.jpg
Saved: face_11.jpg
Saved: face_12.jpg
Total valid images saved: 12


In [ ]:
import os
import cv2
import mediapipe as mp
import numpy as np
from PIL import Image
from io import BytesIO

# Initialize mediapipe face detection and face mesh
mp_face_detection = mp.solutions.face_detection
mp_face_mesh = mp.solutions.face_mesh

# Function to check if all required landmarks are visible
def is_valid_face(image_path):
    """Check if the image has a single, straight-looking face with all landmarks visible."""
    try:
        # Open the image
        image = Image.open(image_path).convert("RGB")
        image_np = np.array(image)  # Convert to NumPy array

        # Initialize Mediapipe face detection
        with mp_face_detection.FaceDetection(min_detection_confidence=0.5) as face_detection:
            results = face_detection.process(cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR))

            # Check if exactly one face is detected
            if not results.detections or len(results.detections) != 1:
                return None  # Invalid image

        # Initialize Mediapipe face mesh for landmarks
        with mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, refine_landmarks=True) as face_mesh:
            results = face_mesh.process(cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR))

            # Check if landmarks are detected
            if not results.multi_face_landmarks:
                return None  # Invalid image

            # Extract landmarks
            landmarks = results.multi_face_landmarks[0]
            landmark_points = landmarks.landmark

            # Ensure all landmarks are detected (we expect 468 landmarks)
            if len(landmark_points) < 468:
                return None  # Not all landmarks detected

            # Check visibility of key facial features: both eyes, nose, and mouth
            left_eye = landmark_points[33]  # Left eye index
            right_eye = landmark_points[263]  # Right eye index
            nose_tip = landmark_points[1]  # Nose tip index
            mouth_left = landmark_points[13]  # Left mouth corner
            mouth_right = landmark_points[14]  # Right mouth corner

            # Check that both eyes are visible
            if left_eye.visibility < 0.5 or right_eye.visibility < 0.5:
                return None  # At least one eye is not visible

            # Check that the nose is visible
            if nose_tip.visibility < 0.5:
                return None  # Nose is not visible

            # Check that the mouth is visible
            if mouth_left.visibility < 0.5 or mouth_right.visibility < 0.5:
                return None  # Mouth is not visible

            # Ensure eyes are horizontally aligned (for a straight face)
            if abs(left_eye.y - right_eye.y) > 0.05:  # Tolerance for alignment
                return None  # Face not straight-looking

            # If all checks pass, return the valid image
            return image

    except Exception as e:
        print(f"Error processing image: {e}")
    return None

# Path to the folder containing images
folder_path = '/content/7'  # Replace with your folder path

# List to store valid images
valid_images = []

# Loop through all images in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(('.png', '.jpg', '.jpeg')):  # Only process image files
        image_path = os.path.join(folder_path, filename)
        print(f"Processing {filename}...")

        valid_image = is_valid_face(image_path)

        if valid_image:
            valid_images.append(valid_image)
            print(f"{filename} is valid.")
        else:
            print(f"{filename} is not valid.")

# Print the number of valid images
print(f"Number of valid images: {len(valid_images)}")


Processing face_1.jpg...
face_1.jpg is not valid.
Processing face_7.jpg...
face_7.jpg is not valid.
Processing face_8.jpg...
face_8.jpg is not valid.
Processing face_4.jpg...
face_4.jpg is not valid.
Processing face_12.jpg...
face_12.jpg is not valid.
Processing face_6.jpg...
face_6.jpg is not valid.
Processing face_11.jpg...
face_11.jpg is not valid.
Processing face_2.jpg...
face_2.jpg is not valid.
Processing face_9.jpg...
face_9.jpg is not valid.
Processing face_3.jpg...
face_3.jpg is not valid.
Processing face_10.jpg...
face_10.jpg is not valid.
Processing face_5.jpg...
face_5.jpg is not valid.
Number of valid images: 0


In [ ]:
import requests
import os
import cv2
import mediapipe as mp
import numpy as np
from PIL import Image
from io import BytesIO

# Initialize mediapipe face detection and face mesh
mp_face_detection = mp.solutions.face_detection
mp_face_mesh = mp.solutions.face_mesh

# Pixabay API key and endpoint
API_KEY = "27542215-1bcc2dba5b7791b910e679866"  # Replace with your Pixabay API Key
PIXABAY_API_URL = "https://pixabay.com/api/"

# Function to download image from a URL
def download_image(image_url):
    """Download image from URL."""
    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content))
    return image

# Function to check if all required landmarks are visible
def is_valid_face(image):
    """Check if the image has a single, straight-looking face with all landmarks visible."""
    try:
        image_np = np.array(image)  # Convert to NumPy array

        # Initialize Mediapipe face detection
        with mp_face_detection.FaceDetection(min_detection_confidence=0.5) as face_detection:
            results = face_detection.process(cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR))

            # Check if exactly one face is detected
            if not results.detections or len(results.detections) != 1:
                return False  # Invalid image

        # Initialize Mediapipe face mesh for landmarks
        with mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, refine_landmarks=True) as face_mesh:
            results = face_mesh.process(cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR))

            # Check if landmarks are detected
            if not results.multi_face_landmarks:
                return False  # Invalid image

            # Extract landmarks
            landmarks = results.multi_face_landmarks[0]
            landmark_points = landmarks.landmark

            # Ensure all landmarks are detected (we expect 468 landmarks)
            if len(landmark_points) < 468:
                return False  # Not all landmarks detected

            # Check visibility of key facial features: both eyes, nose, and mouth
            left_eye = landmark_points[33]  # Left eye index
            right_eye = landmark_points[263]  # Right eye index
            nose_tip = landmark_points[1]  # Nose tip index
            mouth_left = landmark_points[13]  # Left mouth corner
            mouth_right = landmark_points[14]  # Right mouth corner

            # Check that both eyes are visible
            if left_eye.visibility < 0.5 or right_eye.visibility < 0.5:
                return False  # At least one eye is not visible

            # Check that the nose is visible
            if nose_tip.visibility < 0.5:
                return False  # Nose is not visible

            # Check that the mouth is visible
            if mouth_left.visibility < 0.5 or mouth_right.visibility < 0.5:
                return False  # Mouth is not visible

            # Ensure eyes are horizontally aligned (for a straight face)
            if abs(left_eye.y - right_eye.y) > 0.05:  # Tolerance for alignment
                return False  # Face not straight-looking

            # If all checks pass, return valid
            return True

    except Exception as e:
        print(f"Error processing image: {e}")
    return False

# Function to fetch images from Pixabay API
def fetch_images_from_pixabay(query, num_images=50):
    """Fetch images from Pixabay based on search query."""
    params = {
        "key": API_KEY,
        "q": query,
        "image_type": "photo",
        "per_page": num_images
    }
    response = requests.get(PIXABAY_API_URL, params=params)
    data = response.json()

    # Get URLs of the images
    image_urls = [hit["webformatURL"] for hit in data["hits"]]
    return image_urls

# Path to the folder to save valid images
valid_images_folder = '/content/8'
os.makedirs(valid_images_folder, exist_ok=True)

# Fetch images based on search query (e.g., "people face")
image_urls = fetch_images_from_pixabay("Indian face", num_images=50)

valid_image_count = 0

# Process each image URL
for image_url in image_urls:
    print(f"Processing image: {image_url}...")

    # Download the image
    image = download_image(image_url)

    # Check if the image has a valid face
    if is_valid_face(image):
        # Save valid image
        image_name = os.path.join(valid_images_folder, f"valid_image_{valid_image_count + 1}.jpg")
        image.save(image_name)
        print(f"Valid image saved as {image_name}")
        valid_image_count += 1
    else:
        print("Invalid image, discarded.")

# Final output
print(f"Total valid images: {valid_image_count}")


Processing image: https://pixabay.com/get/g171e0869912b85fe7d233bad30c5f619a10cb99cfdd16eac8be6b0cff95379843ae6a415ab6efbcb8039b3602445293f_640.jpg...
Invalid image, discarded.
Processing image: https://pixabay.com/get/g981625ffea2608b1954e662208e26756cbae38a9feed932e76056fe60ace255b37c0ac466915f176aa539977ded8bf76cfa6d33b75da3eda2720d73fc2d45a70_640.jpg...
Invalid image, discarded.
Processing image: https://pixabay.com/get/g7432e0e3967a4079f385d56971999c07d1fcc127cbcb1d2878d5322ec1d6dac54721fd6d57584da52520fc20b74f1d6a_640.jpg...
Invalid image, discarded.
Processing image: https://pixabay.com/get/g0f33986ec9854e1dd9b2e00053b595f75eb7ff6f81364a1f75ea7ebe6d0fae1e666a1f035ad9abebeca97d1cca2985de2305552e71a40a7d301f700535ebdb40_640.jpg...
Invalid image, discarded.
Processing image: https://pixabay.com/get/gb84d90f9747579b4463ad136d6871fcd5aadafc41d253bed286cc3b5d2e0934029cdb20551bd276818a75536677d414af065fd01e973ca49f73c0d484bd193e4_640.jpg...
Invalid image, discarded.
Processing image: h

In [ ]:
os.makedirs("2", exist_ok=True)

In [ ]:
for i, img_data in enumerate(images):
    img_url = img_data.get('webformatURL')
    if img_url:
        valid_image = is_valid_face(img_url)
        if valid_image:
            valid_image.save(f"2/face_{i+1}.jpg")
            print(f"Saved: face_{i+1}.jpg")


Saved: face_1.jpg
Saved: face_2.jpg
Saved: face_3.jpg
Saved: face_7.jpg
Saved: face_8.jpg
Saved: face_9.jpg
Saved: face_10.jpg
Saved: face_17.jpg
Saved: face_19.jpg
Saved: face_20.jpg
Saved: face_23.jpg
Saved: face_24.jpg
Saved: face_25.jpg
Saved: face_26.jpg
Saved: face_31.jpg
Saved: face_32.jpg
Saved: face_34.jpg
Saved: face_35.jpg
Saved: face_37.jpg
Saved: face_41.jpg
Saved: face_44.jpg
Saved: face_46.jpg
Saved: face_47.jpg
Saved: face_48.jpg
Saved: face_49.jpg
Saved: face_50.jpg


In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import os

# Load BLIP model and processor
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Path to the folder containing images
image_dir = "/content/2"  # Update this to the correct directory

# Create a dictionary to store captions
captions = {}

# Loop through each image in the folder
for img_file in os.listdir(image_dir):
    img_path = os.path.join(image_dir, img_file)  # Combine folder path and file name

    # Ensure the path is a valid file and ends with a supported image extension
    if os.path.isfile(img_path) and img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
        try:
            # Open the image
            image = Image.open(img_path)

            # Generate caption
            inputs = processor(images=image, return_tensors="pt")
            outputs = model.generate(**inputs)
            caption = processor.decode(outputs[0], skip_special_tokens=True)

            # Save the caption
            captions[img_file] = caption
            print(f"Caption for {img_file}: {caption}")

        except Exception as e:
            print(f"Error processing {img_file}: {e}")

# Save all captions to a text file
with open("captions.txt", "w") as f:
    for img, caption in captions.items():
        f.write(f"{img}: {caption}\n")


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

Caption for face_20.jpg: a woman sitting on a wall in the street
Caption for face_1.jpg: a woman in a red vest and black shirt
Caption for face_48.jpg: a young girl smiles for the camera
Caption for face_32.jpg: a woman with a pink eye makeup
Caption for face_47.jpg: a woman with yellow face paint
Caption for face_35.jpg: a girl with long hair and bangs
Caption for face_7.jpg: a young boy in a white hat
Caption for face_41.jpg: two young boys sitting on the ground
Caption for face_44.jpg: a bride in a traditional outfit
Caption for face_8.jpg: a bride in a red bridal
Caption for face_25.jpg: a woman in a red dress sitting on a yellow vehicle
Caption for face_46.jpg: a woman in a native costume holding a spear
Caption for face_37.jpg: a young girl in the streets of the old city
Caption for face_23.jpg: a woman in an orange dress standing in front of a blue background
Caption for face_34.jpg: a young girl with fr fr fr fr fr fr fr fr fr fr fr fr fr fr fr
Caption for face_2.jpg: a man in 